In [1]:
# completed segmentation
import torch
import cv2
import numpy as np
from segment_anything import sam_model_registry, SamPredictor
from pyniryo2 import *  # For controlling the Niryo robot and accessing the camera
import pyniryo  # Additional functionalities from pyniryo2

# Load the SAM model
device = "cuda" if torch.cuda.is_available() else "cpu"
sam = sam_model_registry["vit_b"](checkpoint="sam_vit_b_01ec64.pth")
sam.to(device)

# Create predictor
predictor = SamPredictor(sam)

# Set up the ROS instance for the Niryo robot
ros_instance = NiryoRos("169.254.200.200") # Replace with your Niryo robot's IP address
vision = Vision(ros_instance)  # Initialize the vision system

# Function to capture and preprocess the image from the Niryo camera
def capture_image():
    # Fetch the compressed image from the Niryo camera
    img_compressed = vision.get_img_compressed()

    # Get the camera calibration details (intrinsics and distortion coefficients)
    camera_info = vision.get_camera_intrinsics()

    # Uncompress the image to get it ready for processing
    img_uncompressed = pyniryo.uncompress_image(img_compressed)

    # Correct the image for camera distortion using the intrinsics and distortion coefficients
    img = pyniryo.undistort_image(img_uncompressed, camera_info.intrinsics, camera_info.distortion)

    return img

# Capture and prepare the image
image = capture_image()
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Set the image for segmentation
predictor.set_image(image)

# Define the input points as a dictionary
input_points_dict = {
    "p1": (50, 50),
    "p5": (150, 150),
}

# Convert the dictionary values to a NumPy array
input_points = np.array(list(input_points_dict.values()))

# Define corresponding labels for each point (assuming all are positive samples)
input_labels = np.array([1] * len(input_points))  # All points are positive

# Perform segmentation with the first mask output
masks, scores, logits = predictor.predict(input_points, input_labels, multimask_output=False)


# Create a color mask with the same dimensions as the original image
color_mask = np.zeros_like(image)

# Define colors for the masks (for example, blue and green)
colors = [
    (0, 255, 0),  # Green for chip 1
    (255, 0, 0),  # Blue for chip 2
    (0, 0, 255),  # Red for chip 3
    # You can define more colors if you have more chips
]

# If masks are generated, apply colors
if len(masks) > 0:
    for i, mask in enumerate(masks):
        color = colors[i % len(colors)]  # Cycle through the defined colors
        color_mask[mask.astype(bool)] = color  # Assign color to the mask area

# Overlay the color mask on the original image
overlayed_image = cv2.addWeighted(image, 0.5, color_mask, 0.5, 0)

# Save the colored mask and overlayed image
cv2.imwrite('colored_mask.png', color_mask)  # Save the colored mask
cv2.imwrite('overlayed_image.png', overlayed_image)  # Save the overlayed image

# Optionally display the images
cv2.imshow('Colored Mask', color_mask)
cv2.imshow('Overlayed Image', overlayed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

c:\Users\kashi\AppData\Local\Programs\Python\Python312\Lib\site-packages\segment_anything\build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.

True